In [1]:
from huggingface_hub import notebook_login
notebook_login()

# Diarization

In [7]:
import os
import librosa
import IPython.display
from subprocess import CalledProcessError, run
from pyannote.audio import Pipeline

INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
C:\Users\jongb\AppData\Local\Programs\Python\Python312\Lib\inspect.py:1007: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):


In [ ]:
def split_audio(input_file, output_file, start, end):
    length = end - start
    cmd = ["ffmpeg", "-ss", str(start), "-i", input_file, "-t", str(length), "-vn", "-acodec", "pcm_s16le", "-ar", "48000", "-ac", "1", output_file]
    try:
        run(cmd, capture_output=True, check=True).stdout
    except CalledProcessError as e:
        raise RuntimeError(f"FFMPEG error {str(e)}")
        


In [4]:
input_wav = "./audio_files/test01.wav"
output_dir = "output"
count = 10001


d:\AA_develop\Diarization\myenv\Lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1823.)
  std = sequences.std(dim=-1, correction=1)


start=0.38534375s stop=5.599718750000001s speaker_SPEAKER_01
start=6.578468750000001s stop=15.083468750000002s speaker_SPEAKER_00


In [ ]:
# inference
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1")
diarization = pipeline(input_wav)
diarization

In [ ]:
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"start={turn.start}s stop={turn.end}s speaker_{speaker}")

    speaker_dir = f"{output_dir}/speaker_{speaker}/"
    if not os.path.isdir(speaker_dir):
        os.mkdir(speaker_dir)

    filename = os.path.join(speaker_dir, f"interview-{count}.wav")
    split_audio(input_wav, filename, turn.start, turn.end)
    count += 1

In [11]:
audio_path = "./audio_files/test01.wav"
audio, sr = librosa.load(audio_path, sr=16000)
IPython.display.Audio(data=audio, rate=sr)

In [8]:
audio_path = './output/speaker_SPEAKER_01/interview-10001.wav'
audio, sr = librosa.load(audio_path, sr=16000)
IPython.display.Audio(data=audio, rate=sr)

In [9]:
audio_path = './output/speaker_SPEAKER_00/interview-10002.wav'
audio, sr = librosa.load(audio_path, sr=16000)
IPython.display.Audio(data=audio, rate=sr)

# Wav to Text

In [4]:
import speech_recognition as sr# Initialize recognizer class                                       
r = sr.Recognizer()# audio object                                                         
audio = sr.AudioFile("./audio_files/test01.wav")#read audio object and transcribe
with audio as source:
    audio = r.record(source)                  
    result = r.recognize_google(audio, language='ko-KR')
    
print(result)

현대자동차가 미국 대통령 취임식에 처음으로 기부금을 했다 도널드 트럼프 미국 대통령 당선인과의 우호적인 관계를 맺고자 하는 글로벌 완성차 업체들의 기부 행렬에 동참한 것으로 풀이된다


In [5]:
import speech_recognition as sr# Initialize recognizer class                                       
r = sr.Recognizer()# audio object                                                         
audio = sr.AudioFile("./output/speaker_SPEAKER_01/interview-10001.wav")#read audio object and transcribe
with audio as source:
    audio = r.record(source)                  
    result = r.recognize_google(audio, language='ko-KR')
    
print(result)

현대자동차가 미국 대통령 취임식에 처음으로 기기 구매했다


In [2]:
import speech_recognition as sr# Initialize recognizer class                                       
r = sr.Recognizer()# audio object                                                         
audio = sr.AudioFile("./output/speaker_SPEAKER_00/interview-10002.wav")#read audio object and transcribe
with audio as source:
    audio = r.record(source)                  
    result = r.recognize_google(audio, language='ko-KR')
    
print(result)

도널드 트럼프 미국 대통령 당선인과의 우호적인 관계를 맺고자 하는 글로벌 완성차 업체들의 기부 행렬에 동참한 것으로 풀이된다
